In [1]:
import sqlite3
import sys
import datetime

In [2]:
#Создаем новую БД, подключаемся к ней, создаем таблицу сотрудников
connection = sqlite3.connect('ndb.db', timeout=100)
cursor = connection.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS emp (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
depar VARCHAR(255) NOT NULL,
num INTEGER,
dates TEXT,
vacation TEXT
)
''')
connection.commit()

In [3]:
# Добавляем данные в таблицу
cursor.execute('''
INSERT INTO emp 
('name', 'depar', 'num', 'dates', 'vacation')
VALUES
('Иванов', 'SDH, PDH, WDM', '3', '2024-02-02', 'с 2024-02-05 - 2024-02-15'),
('Петров', 'СПД, Сервера', '6', '2024-01-30', 'с 2024-02-07 - 2024-02-09'),
('Сидоров', 'ОбТС, PDH', '3', '2024-02-02', '-'),
('Рабинович', 'Сервера, WDM', '5', '2024-01-30', 'с 2024-02-12 - 2024-02-15'),
('Зозо', 'ОбТС, СПД', '3', '2024-01-31', 'с 2024-02-07 - 2024-02-09')
''')
connection.commit()

In [4]:
#Спрашиваем у пользователя, какое направление для создания заявки
dep1 = input("Введите нарпавление: SDH, PDH, WDM, СПД, Сервера, ОбТС ")
dep = "%"+dep1+"%"
fixd1 = '2024-02-02'
fixd = datetime.datetime.strptime(fixd1, "%Y-%m-%d").date()
#Спрашиваем у пользователя дату для создания заявки, ввести можно только в одном формате YYYY-MM-DD
dt1 = input("Введите дату в формате год-месяц-день, например, 2024-02-02 ")
dt = datetime.datetime.strptime(dt1, "%Y-%m-%d").date()
if dt < fixd:
    print("Вы ввели дату меньше, чем 2024-02-02 ")
    while dt < fixd:
        dt12 = input("Введите дату более 2024-02-01 ")
        dt = datetime.datetime.strptime(dt12, "%Y-%m-%d").date()
print("Вы ввели: ", dep, dt)

Введите нарпавление: SDH, PDH, WDM, СПД, Сервера, ОбТС Сервера
Введите дату в формате год-месяц-день, например, 2024-02-02 2024-02-08
Вы ввели:  %Сервера% 2024-02-08


In [5]:
#Создаем текст запроса, делаем запрос, получаем данные.
sql = '''
SELECT *, 
TRIM(substr(vacation, 1, instr(vacation, ' - ') -1), 'с ') AS vac_start, 
substr(vacation, instr(vacation, ' - ') +3) AS vac_end
FROM(SELECT * FROM emp WHERE depar LIKE '%s')
WHERE (vac_start > ? OR ? > vac_end) OR vacation = '-'
ORDER BY num, dates
LIMIT 1; 
''' %dep
cursor.execute(sql, (dt, dt))
la = cursor.fetchall()

In [6]:
#Выводим на экран фамилию сотрудника,который может взять заявку
if len(la)==0:
        print("Нет свободных сотрудников")
else:
    out = la[0]
    print(out)
    print("Вы можете создать заявку на сотрудника ", out[1])
    

(4, 'Рабинович', 'Сервера, WDM', 5, '2024-01-30', 'с 2024-02-12 - 2024-02-15', '2024-02-12', '2024-02-15')
Вы можете создать заявку на сотрудника  Рабинович


In [7]:
#Закрываем подключение
cursor.close()
connection.close()